In [46]:
import pandas as pd
import os
import sqlite3
from sqlite3 import Error

In [47]:
S_data = pd.read_csv(r'C:\Data Analysis - Files\Project 5 - Superstore\Sample-Superstore.csv', sep=',', encoding='latin1', on_bad_lines='skip')

In [48]:
S_data = S_data.drop(columns=['Row ID'])
S_data = S_data.rename(columns={'Order ID': 'Order_ID', 'Customer ID': 'Customer_ID', 'Product ID': 'Product_ID'})

In [49]:
#Creating an empty sqlite database to populate with the newly created CSVs
conn = sqlite3.connect("superstore_database.db")

In [50]:
# Establishing a cursor object (a tool that executes SQL code against the database)
c = conn.cursor()

In [51]:
#create the (empty) tables which will be included in the database.

#Product_Details

c.execute('''
CREATE TABLE IF NOT EXISTS "Product_Details" (
	"Product_ID"	CHAR(20) NOT NULL,
	"Category"	VARCHAR(30) NOT NULL,
	"SubCategory"	VARCHAR(30) NOT NULL,
    "Product Name"	VARCHAR(30) NOT NULL,
	PRIMARY KEY("Product_ID")
);
''')

c.execute('''
CREATE TABLE IF NOT EXISTS "Customer_Details" (
	"Customer_ID"	CHAR(30) NOT NULL,
	"Customer Name"	VARCHAR(30) NOT NULL,
	"Segment"	VARCHAR(15) NOT NULL,
	PRIMARY KEY("Customer_ID")
);
''')

c.execute('''
CREATE TABLE IF NOT EXISTS "Address_Details" (
	"Address_ID" INTEGER PRIMARY KEY AUTOINCREMENT,
  	"Country" text NOT NULL,
  	"City" text NOT NULL,
  	"State" text NOT NULL,
  	"PostalCode" integer NOT NULL,
  	"Region" text NOT NULL
);
''')

c.execute('''
CREATE TABLE IF NOT EXISTS "Order_Details" (
	"Order_ID"	CHAR(20) NOT NULL,
	"Order Date" date NOT NULL,
    "Ship Date" date NOT NULL, 
    "Ship Mode" varchar NOT NULL,
    "Customer_ID" varchar NOT NULL,
	PRIMARY KEY("Order_ID"),
	FOREIGN KEY(Customer_ID) REFERENCES Customer_Details(Customer_ID)
);
''')

c.execute('''
CREATE TABLE IF NOT EXISTS "Product_Sales" (
	"Sales" integer NOT NULL,
    "Quantity"  integer NOT NULL,
    "Discount"  integer NOT NULL,
    "Profit"  integer NOT NULL,
    "Order_ID" varchar NOT NULL,
    "Product_ID" varchar NOT NULL,
    FOREIGN KEY(Order_ID) REFERENCES Order_Details(Order_ID),
    FOREIGN KEY(Product_ID) REFERENCES Product_Details(Product_ID)
);
''')

In [52]:
# If the table already exists and you want to append data:
S_data_order = S_data [["Order_ID","Order Date","Ship Date","Ship Mode","Customer_ID"]]
S_data_order = S_data_order.drop_duplicates(subset=['Order_ID'])
S_data_order.to_sql('Order_Details', conn, if_exists='append', index=False)

S_data_Customer = S_data [["Customer_ID","Customer Name","Segment"]]
S_data_Customer = S_data_Customer.drop_duplicates(subset=['Customer_ID'])
S_data_Customer.to_sql('Customer_Details', conn, if_exists='append', index=False)

# S_data.to_sql('Address_Details', conn, if_exists='append', index=False)
# S_data.to_sql('Product_Details', conn, if_exists='append', index=False)
# S_data.to_sql('Product_Sales', conn, if_exists='append', index=False)

# Step 5: Commit and close the connection
conn.commit()
conn.close()

IntegrityError: UNIQUE constraint failed: Order_Details.Order_ID